Goals
1. Download all zippped files and unzip csv files to 2 seperate folders. One for Ford and one for Lyft - Do in automated fashion if possible
2. Combine each of the 10+ csv files into 2 big files, one for Ford and one for Lyft
3. Push to AWS S3

Bonus: Each month write a code that auto pulls the new zipped file from the cloud and appends it to our existing Lyft AWS dataset

In [3]:
#I will utilize some code from my past web scraping project and tweak it for this project.
#I will use selenium since that is what I have used in the past and is what I am familiar with

#This currently looks up certain fields for you and enters them into the website. 
#Next would be to automate this to run through all 4 designations I want. Do that at end. Also make cleaner later.
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
from tabulate import tabulate
import os
import math
import regex as re 
from selenium.webdriver.support.ui import Select

from zipfile import ZipFile
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from decimal import Decimal
import numpy as np
import pandas as pd
import math
import os
import io

from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.metrics import r2_score
import boto3 


In [ ]:
#This will auto download all the code currently on baywheels website

#set download path
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : r"C:\Users\gadde\Desktop\Data_Analytics_projects\baywheels_analysis\zipped_data"}
chromeOptions.add_experimental_option("prefs",prefs)

#start chromedriver
driver = webdriver.Chrome(chrome_options=chromeOptions)
driver.get('https://s3.amazonaws.com/baywheels-data/index.html')
#//*[@id="tbody-content"]/tr[1]/td[1]/a
time.sleep(1)

#Note we have it hard set to 53. Ideally we could use a lenght of something to automatically determine this but I don't
#know how to do that.
for i in range(53):
    download = driver.find_element(by=By.XPATH, value='//*[@id="tbody-content"]/tr[' + str(i+1) + ']/td[1]/a')
    download.click()

#Close out driver
driver.quit()

In [4]:
#now let us go ahead and unzip all the files in the zipped_data folder
for root, dirs, files in os.walk('zipped_data'):
    for name in files:
        if 'ford' in name:
            with ZipFile(os.path.join(root, name), 'r') as zip_ref:
                zip_ref.extractall('baywheels_data\Ford')
        elif 'baywheels' in name:
            with ZipFile(os.path.join(root, name), 'r') as zip_ref:
                zip_ref.extractall('baywheels_data\Lyft')
        else:
            print('Error')

In [6]:
#Now lets combine the multiple csv's in each of these folders into one large df and output that to a csv
path = 'baywheels_data\Ford'
ford_file_names = []
for files in os.listdir(path):
    if 'csv' in files:
        ford_file_names.append(os.path.join(path, files))
        #print(os.path.join(path, files))
        
#Load the csv into a dataframe
df_ford = pd.concat(map(pd.read_csv, ford_file_names))


In [7]:
#repeat above step for Lyft data
path = 'baywheels_data\Lyft'
lyft_file_names = []
for files in os.listdir(path):
    if 'csv' in files:
        lyft_file_names.append(os.path.join(path, files))
        #print(os.path.join(path, files))
        
#Load the csv into a dataframe
df_lyft = pd.concat(map(pd.read_csv, lyft_file_names))

C:\Users\gadde\anaconda3\lib\site-packages\pandas\core\reshape\concat.py:294: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  op = _Concatenator(
C:\Users\gadde\anaconda3\lib\site-packages\pandas\core\reshape\concat.py:294: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  op = _Concatenator(
C:\Users\gadde\anaconda3\lib\site-packages\pandas\core\reshape\concat.py:294: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  op = _Concatenator(


In [10]:
#It seems to be missing a lot of starting times when combining the whole lyft dataset but NOT when pulling each excel file ind.
#Try to troubleshoot this problem
df_lyft_subpart = pd.read_csv('baywheels_data/Lyft/202204-baywheels-tripdata.csv')
df_lyft_subpart.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
218225,BB6FD59915844CCA,electric_bike,2022-04-16 13:42:04,2022-04-16 14:17:45,NaN,NaN,Valencia St at 21st St,SF-P22,37.750000,-122.500000,37.756708,-122.421025,member
218226,FB9DADA6640ABD3C,electric_bike,2022-04-26 19:14:56,2022-04-26 19:19:09,NaN,NaN,Mission Playground,SF-O22,37.760000,-122.410000,37.759210,-122.421339,member
218227,4B4FB26BC22EB13A,electric_bike,2022-04-13 18:49:24,2022-04-13 19:02:47,Powell St BART Station (Market St at 4th St),SF-G27,Mission Playground,SF-O22,37.786300,-122.405021,37.759210,-122.421339,casual
218228,574F25274592A3F1,classic_bike,2022-04-01 17:32:58,2022-04-01 17:34:27,Harrison St at 17th St,SF-N24-2,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.763847,-122.413004,37.763735,-122.415524,member
218229,CDA0A4687BD27127,electric_bike,2022-04-18 17:41:45,2022-04-18 18:05:45,Arguello Blvd at Edward St,SF-H13,Valencia St at 21st St,SF-P22,37.778636,-122.458652,37.756708,-122.421025,casual


In [ ]:
#Write the ford and lyft data to a csv
df_ford.to_csv(r'baywheels_data\ford_data.csv')
df_lyft.to_csv(r'baywheels_data\lyft_data.csv')

In [ ]:
#Lastly, let us upload these 2 large dataframes in an S3 bucket. We will upload this using the csv format.
#I know that are probably more efficient file storage formats but since this data isn't huge csv should suffice

s3 = boto3.client("s3")

s3.upload_file(
    Filename=r'baywheels_data\ford_data.csv',
    Bucket="baywheels.data",
    Key="ford_data.csv",
)

In [ ]:
#Same for Lyft data

s3 = boto3.client("s3")

s3.upload_file(
    Filename=r'baywheels_data\lyft_data.csv',
    Bucket="baywheels.data",
    Key="lyft_data.csv",
)